## Initial Setup

The setup below shows in-code configuration. For most applications, we can also enable LLMObs simply by calling `ddtrace-run` with the appropriate env vars, [as seen here in our quickstart instructions](https://docs.datadoghq.com/tracing/llm_observability/quickstart/).

The code below requires you to have already created an `.env` file with several configuration variables as explained in the README.


In [1]:
from dotenv import load_dotenv

load_dotenv()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

**Note for enterprise customers using secrets:**

If you are using secrets, you can enable LLM Observability with more specific parameters as demonstrated below.

```python
LLMObs.enable(
  ml_app="<YOUR_ML_APP_NAME>",
  api_key="<YOUR_DATADOG_API_KEY>",
  site="<YOUR_DATADOG_SITE>",
  agentless_enabled=True,
)
```


## Tracing an LLM Call

An LLM Span represents a call to a model. In this simple example, we are asking `gpt-3.5-turbo` to summarize a provided text and identify a list of topics from the text.

Because we use OpenAI, the call to the LLM is instrumented automatically by Datadog, with no further action required on our part:


In [2]:
from openai import OpenAI
import json
import os

oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


sys_prompt = """
Your task is to 
1. Summarize the given text at a 6th grade reading level in no more than 2 sentences.
2. Identify what topics the text belongs to that would allow you to categorize it in a school library.
Format your output strictly following this JSON convention:
{	
    "topics": <[insert array of topics here]>
    "summary": <insert summary here>
}
	"""


def summarize(text, prompt=sys_prompt):
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": text},
    ]
    # llm span auto-instrumented via our openai integration
    response_content = (
        oai_client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
        )
        .choices[0]
        .message.content
    )
    return json.loads(response_content)

In [3]:
text = """
ONE JANUARY day, thirty years ago, the little town of Hanover, anchored on a windy Nebraska tableland, was trying not to be blown away. A mist of fine snowflakes was curling and eddying about the cluster of low drab buildings huddled on the gray prairie, under a gray sky. The dwelling-houses were set about haphazard on the tough prairie sod; some of them looked as if they had been moved in overnight, and others as if they were straying off by themselves, headed straight for the open plain. None of them had any appearance of permanence, and the howling wind blew under them as well as over them. The main street was a deeply rutted road, now frozen hard, which ran from the squat red railway station and the grain "elevator" at the north end of the town to the lumber yard and the horse pond at the south end. On either side of this road straggled two uneven rows of wooden buildings; the general merchandise stores, the two banks, the drug store, the feed store, the saloon, the post-office. The board sidewalks were gray with trampled snow, but at two o'clock in the afternoon the shopkeepers, having come back from dinner, were keeping well behind their frosty windows.
"""

In [4]:
summarize(text)

{'topics': ['History', 'Rural Life', 'Weather Phenomena'],
 'summary': "In a small Nebraska town thirty years ago, the residents struggle against harsh winter weather conditions as fine snowflakes swirl around the low buildings. The town's main street, lined with various stores and businesses, is quiet in the afternoon as the shopkeepers stay inside to keep warm."}

## Viewing the trace in Datadog

Now, check out the [LLM Observability interface](https://app.datadoghq.com/llm) in Datadog. You should see a trace that describes the LLM call, including the system prompt, the user prompt, and the response.


#### Additional resources

- [List of all integrations supported by Datadog's LLM Observability product](https://docs.datadoghq.com/tracing/llm_observability/sdk/#llm-integrations)
- [Instructions for manually instrumenting an LLM Span](https://docs.datadoghq.com/tracing/llm_observability/sdk/#llm-span)
